In [19]:
import numpy as np
import cvxpy as cp
import tensorflow as tf
from cvxpylayers.tensorflow import CvxpyLayer

n, m = 3, 1
M = np.array([[1., 2., 0.], [-8., 3., 2.], [0., 1., 1.]])
x = cp.Variable(n)
Q = np.dot(M.T, M) #cp.Parameter((n,n))
q = np.dot(np.array([3., 2., 3.]), M)
G = cp.Parameter(n,n)
h = np.array([3., 2., -2.]).reshape((3,))
A = np.array([1., 1., 1.])
b = np.array([1.])
constraints = [G@x <= h, A@x == b]
objective = cp.Minimize(0.5*cp.quad_form(x, Q) + q.T @ x)
problem = cp.Problem(objective, constraints)
assert problem.is_dpp()

cvxpylayer = CvxpyLayer(problem, parameters=[h], variables=[x])

# _Q = tf.Variable(dot(M.T, M))  # quick way to build a symmetric matrix
_G = tf.Variable([[1., 2., 1.], [2., 0., 1.], [-1., 2., -1.]])

with tf.GradientTape() as tape:
  # solve the problem
  solution, = cvxpylayer(_Q, _G)
    
  summed_solution = tf.math.reduce_sum(solution)

# compute the gradient of the summed solution with respect to params
gradh = tape.gradient(summed_solution, [_Q, _G])

TypeError: unhashable type: 'numpy.ndarray'

In [17]:
problem

Problem(Minimize(Expression(UNKNOWN, UNKNOWN, (1, 1))), [Inequality(Expression(AFFINE, UNKNOWN, (3,))), Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONNEGATIVE, (1,)))])

In [10]:
solution

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.33311126, 0.33311126])>

## diff wrt 2 variables h[1:2]

In [59]:
from qpth.qp import QPFunction
import torch
from torch import nn
from torch.autograd import Variable

In [95]:
Q = np.array([[4, 1], [1, 2]])
q = np.array([1, 1])
G = np.array([[2, 1], [1, 2]])
h = np.array([-1., -1.])

In [96]:
q, Q, G, h = [torch.Tensor(x) for x in [q, Q, G, h]]

A, b = [torch.Tensor()] * 2

q, Q, G, h, A, b = [Variable(x) for x in [q, Q, G, h, A, b]]
h.requires_grad = True

In [97]:
qpf = QPFunction()
zhat_b = qpf(Q, q, G, h, A, b)

In [98]:
zhat_b  # solution

tensor([[-0.2500, -0.5000]], grad_fn=<QPFunctionFnBackward>)

In [99]:
zhat_b.backward(h.unsqueeze(0))

In [100]:
grads = [x.grad.data.squeeze(0).cpu().numpy() for x in [h]]

In [101]:
# actually dho z / dho h should be 2x2 matrix (refer the julia script)
# but qpth and cvxpylayers dont return these actual jacobians
# rather they return their partial sums
# here grads is [dz[1]/dh[1] + dz[2]/dh[1], dz[1]/dh[2], dz[2]/dh[2]]

grads

[array([-6.2500000e-01, -1.5287696e-08], dtype=float32)]